In [1]:
from util import *

2023-12-30 22:12:26.312376: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 22:12:26.312598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 22:12:26.325800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 22:12:26.403998: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 22:12:28.301285: W tensorflow/compiler/tf2

In [2]:
EXTRACT_FEATRURES = False

In [3]:
train_dir = 'Dataset/train_clean.txt'
val_dir = 'Dataset/val_clean.txt'
test_dir = 'Dataset/test_clean.txt'

In [4]:
with open(train_dir, 'r', encoding='utf8') as f:
    train = f.read()
with open(val_dir, 'r', encoding='utf8') as f:
    val = f.read()
with open(test_dir, 'r', encoding='utf8') as f:
    test = f.read()

In [5]:
max_len = 500

In [6]:
X_train_dir = 'pkl_dir/X_train.txt'
y_train_dir = 'pkl_dir/y_train.txt'
X_val_dir = 'pkl_dir/X_val.txt'
y_val_dir = 'pkl_dir/y_val.txt'
X_test_dir = 'pkl_dir/X_test.txt'
y_test_dir = 'pkl_dir/y_test.txt'

if EXTRACT_FEATRURES:
    X_train, y_train = extract_features(train, max_len)
    X_val, y_val = extract_features(val, max_len)
    X_test, y_test = extract_features(test, max_len)

    with open(X_train_dir, 'wb') as f:
        pickle.dump(X_train, f)
    with open(y_train_dir, 'wb') as f:
        pickle.dump(y_train, f)
    with open(X_val_dir, 'wb') as f:
        pickle.dump(X_val, f)
    with open(y_val_dir, 'wb') as f:
        pickle.dump(y_val, f)
    with open(X_test_dir, 'wb') as f:
        pickle.dump(X_test, f)
    with open(y_test_dir, 'wb') as f:
        pickle.dump(y_test, f)

else:
    with open(X_train_dir, 'rb') as f:
        X_train = pickle.load(f)
    with open(y_train_dir, 'rb') as f:
        y_train = pickle.load(f)
    with open(X_val_dir, 'rb') as f:
        X_val = pickle.load(f)
    with open(y_val_dir, 'rb') as f:
        y_val = pickle.load(f)
    with open(X_test_dir, 'rb') as f:
        X_test = pickle.load(f)
    with open(y_test_dir, 'rb') as f:
        y_test = pickle.load(f)

In [7]:
j = 0
i = 20
print(chr(X_train[j][i]))
print(id2diacritic[y_train[j][i]])

و
َّ


In [8]:
del train, val, test

In [9]:
test_len = 10000
np.random.seed(42)
indices = np.arange(test_len)
np.random.shuffle(indices)
X_test = X_test[indices]
y_test = y_test[indices]

In [32]:
sentence_encoder = LabelEncoder().fit(X_train.flatten())
X_train = sentence_encoder.transform(X_train.flatten()).reshape(X_train.shape).astype(np.uint8)
X_val = sentence_encoder.transform(X_val.flatten()).reshape(X_val.shape).astype(np.uint8)
X_test = sentence_encoder.transform(X_test.flatten()).reshape(X_test.shape).astype(np.uint8)

In [33]:
# from keras.models import Sequential
# from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, BatchNormalization, Dropout, TimeDistributed, Dense

# model = Sequential()

# model.add(Embedding(input_dim=np.unique(X_train).shape[0], output_dim=25, input_length=max_len))

# model.add(Conv1D(filters=256, kernel_size=10, activation='relu', padding='same'))
# model.add(MaxPooling1D(pool_size=2, padding='same'))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.5))

# model.add(Bidirectional(LSTM(256, return_sequences=True)))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.5))

# model.add(Bidirectional(LSTM(256, return_sequences=True)))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.5))

# model.add(TimeDistributed(Dense(512, activation='relu')))
# model.add(BatchNormalization())

# model.add(TimeDistributed(Dense(512, activation='relu')))
# model.add(BatchNormalization())

# model.add(TimeDistributed(Dense(np.unique(y_train).shape[0], activation='softmax')))

# model.summary()


# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model = tf.keras.models.load_model('model_conv.h5')

In [34]:
history = model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=30, batch_size=80, validation_data=(X_val, tf.keras.utils.to_categorical(y_val)))

Epoch 1/30
500/500 [==============================] - 137s 260ms/step - loss: 0.0456 - accuracy: 0.9845 - val_loss: 0.0375 - val_accuracy: 0.9874
Epoch 2/30
 57/500 [==>...........................] - ETA: 1:57 - loss: 0.0437 - accuracy: 0.9852

KeyboardInterrupt: 

In [35]:
model.save('model_conv.h5')

/home/john/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=2)
ignore = {'!', '«', ']', '[', '}', ':', '"', '-', '»', '؛', ')', '،', '؟', '(', '{', '/', ' ', PAD, SOS, EOS}
cnt = 0
for itm in ignore:
    cnt += np.sum(X_test == sentence_encoder.transform(np.array([ord(itm)]))[0])
accuracy = (np.sum(predictions == y_test) - cnt) / (predictions.shape[0] * predictions.shape[1] - cnt)
print('DER: ', 1 - accuracy)

313/313 [==============================] - 25s 74ms/step
DER:  0.03762377428863628


In [ ]:
sentence_test_str = 'كَانَ مَعَهُ أَلْفٌ فَقَالَ هِيَ مُضَارَبَةٌ لِفُلَانٍ بِالنِّصْفِ وَقَدْ رَبِحَ أَلْفًا فَقَالَ فُلَانٌ هِيَ بِضَاعَةٌ فَالْقَوْلُ قَوْلُ رَبِّ الْمَالِ'
print(sentence_test_str)
print(diacritize_string(sentence_test_str, model, sentence_encoder, max_len)[1:-1])

كَانَ مَعَهُ أَلْفٌ فَقَالَ هِيَ مُضَارَبَةٌ لِفُلَانٍ بِالنِّصْفِ وَقَدْ رَبِحَ أَلْفًا فَقَالَ فُلَانٌ هِيَ بِضَاعَةٌ فَالْقَوْلُ قَوْلُ رَبِّ الْمَالِ
1/1 [==============================] - 0s 273ms/step
كَانَ مَعَهُ أَلْفٌ فَقَالَ هِيَ مُضَارَبَةٌ لِفُلَانٍ بِالنِّصْفِ وَقَدْ رَبِحَ أَلْفًا فَقَالَ فُلَانٌ هِيَ بِضَاعَةٍ فَالْقَوْلُ قَوْلُ رَبِّ الْمَالِ
